In [2]:
import os
import numpy as np
import pandas as pd
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

In [3]:
file1 = open('../../datasets/AAPD/keyword_ext/update_labelspace_pke.txt', 'r')
documents = file1.readlines()  
predict_label_space = []
for doc in documents:
    predict_label_space.append(doc.strip())

In [125]:
rep_label = []
for i in range(len(df)):
    s = df['Representation'][i]
    if not s[1:-1].split(',')[0][1:-1] in rep_label:
        rep_label.append(s[1:-1].split(',')[0][1:-1])
predict_label_space = rep_label

In [126]:
file2 = open('../../datasets/Amazon-531/train/labels.txt', 'r')
labels = file2.readlines()
true_label = []
for row in labels:
    true_label.append(row.strip().split('\t')[1])
true_label[:5]

['grocery_gourmet_food', 'meat_poultry', 'jerky', 'toys_games', 'games']

In [4]:
file2 = open('../../datasets/AAPD/true_labels.txt', 'r')
labels = file2.readlines()
label_dic = {}
for row in labels:
    key = row.strip().split(';')[0].split('.')[0]
    #key + '.' + 
    label_dic[row.strip().split(';')[0]] = row.strip().split(';')[1][1:]
true_label = []
for key in label_dic:
    if not label_dic[key] in true_label:
        true_label.append(label_dic[key])
true_label[:5]

['Information Retrieval',
 'Methodology',
 'Quantum Physics',
 'Information Theory',
 'Applications']

In [ ]:
file2 = open('../../datasets/RCV1-V2/rcv1.topics.hier.orig.txt', 'r')
labels = file2.readlines() 
true_label = []
for row in labels[1:]:
    new_label = row.strip().split('child-description: ')[1].lower()
    true_label.append(new_label)
true_label[:5]

In [113]:
file2 = open('../../datasets/Reuters-21578/train_label.txt', 'r')
labels = file2.readlines()
true_label = []
for row in labels:
    label_list = row.strip().split(' ')
    for label in label_list:
        if not label in true_label:
            true_label.append(label)
true_label[:5]

['cocoa', 'sorghum', 'oat', 'barley', 'corn']

In [5]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

query_embedding = model.encode('teethers')
passage_embedding = model.encode(['baby_dental_care'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.4893]])


In [6]:
passage_embedding = model.encode(predict_label_space)

In [7]:
sim_matrix = np.empty((0,len(predict_label_space)))
for i in range(len(true_label)):
    query_embedding = model.encode(true_label[i])
    sim_matrix = np.append(sim_matrix, util.dot_score(query_embedding, passage_embedding).numpy(), 0)

In [8]:
sim_matrix.shape

(49, 77)

In [9]:
sim_list = []
for i, sim_score in enumerate(sim_matrix):
    rank_list = np.argsort(sim_score)[-40:]
    cur_list = [true_label[i]]
    for index in rank_list:
        cur_list.append((sim_score[index], predict_label_space[index]))
    if cur_list[-1][0] >= 0.6:
        sim_list.append(cur_list)

In [10]:
pair_list = []
for line in sim_list:
    true_label = line[0]
    for row in line[1:]:
        if row[0] >= 0.6:
            pair_list.append([(true_label, row[1]), row[0]])
sorted_pair_list = sorted(pair_list, key=lambda x: x[1], reverse = True)
sorted_pair_list

[[('Probability', 'probability'), 1.0],
 [('Optimization and Control', 'optimal control'), 0.8559101819992065],
 [('Numerical Analysis', 'extensive numerical investigations'),
  0.8089126348495483],
 [('Statistics and Probability', 'probability'), 0.7721471786499023],
 [('Quantum Physics', 'quantum computations'), 0.7185327410697937],
 [('Performance', 'performance final comparisons'), 0.6822025179862976],
 [('Performance', 'efficiency'), 0.6781799793243408],
 [('Computational Complexity', 'sample complexity'), 0.6666020154953003],
 [('Optimization and Control', 'convex optimization'), 0.6558310985565186],
 [('Cryptography and Security', 'secure computation'), 0.633407473564148],
 [('Robotics', 'autonomous driving'), 0.6323422193527222],
 [('Social and Information Networks', 'networks'), 0.6213299632072449],
 [('Logic', 'finite logical implication'), 0.6042479276657104]]

In [11]:
refine_pair_list = []
while len(sorted_pair_list) > 0:
    refine_pair_list.append([sorted_pair_list[0][0], sorted_pair_list[0][1]])
    ground_truth = sorted_pair_list[0][0][0]
    prediction = sorted_pair_list[0][0][1]
    sorted_pair_list.pop(0)
    delete_list = []
    for row in sorted_pair_list:
        if row[0][0] == ground_truth or row[0][1] == prediction:
            delete_list.append(row)
    for item in delete_list:
        sorted_pair_list.remove(item)
refine_pair_list[:5]

[[('Probability', 'probability'), 1.0],
 [('Optimization and Control', 'optimal control'), 0.8559101819992065],
 [('Numerical Analysis', 'extensive numerical investigations'),
  0.8089126348495483],
 [('Quantum Physics', 'quantum computations'), 0.7185327410697937],
 [('Performance', 'performance final comparisons'), 0.6822025179862976]]

In [12]:
len(refine_pair_list)

10

In [13]:
10/49

0.20408163265306123